# JPEG Compression

- Valérian FAYT <valerian.fayt>
- Romain HERMARY <romain.hermary>
- Quentin LE HELLOCO <quentin.le-helloco>

https://web.microsoftstream.com/video/6d64a720-a41c-4680-ae5c-dfd27fb36be6

https://web.microsoftstream.com/video/588e5cd6-1209-48f6-991a-341b862279ce

## Project

#### In this notebook, you have to:
* Illustrate and comment every different step of your algorithm, like if you had to explain it to someone who never heard of JPEG.
* Implement and analyses all relevant tests to demonstrate the proper functioning of the algorithm.

#### Your JPEG code should at least:
* Manage color, leaving the choice to the user to compress in RGB or YUV, as well as the sub-sampling options (4:4:4, 4:2:2 and 4:2:0) of chrominance.
* Manage images whose dimensions are not 8 multiples
* Let the user choose the quality indicator q for the luminance quantification matrix.
* Return for each macro-block, a compression indicator. This indicator may be define as in the course (number of coefficient by macro-block to code without compression (64) divide by the number of coefficient not null after zigzag linearisation of the DCT quantified matrix). It can be any other relevant indicator as well, if the choice is justified.

#### Bonus:
* Implement the conversion of DCT coefficients after quantification by Huffman table.

### JPEG Algorithm

* Step 1 :
Split images in 8x8 blocks (if not multiple, add new columns/row with symmetrical inputs.

* Step 2 :
Apply DCT matrix (don't forget to -128)

* Step 3 :
Quantification of the matrix

* Step 4 :
Zigzag + Huffman compression

(Reversible to uncompress JPEG as well)

### Import

In [ ]:
import numpy as np
from PIL import Image

### Get list of pixel from Image

In [ ]:
im = Image.open('Images/amidala_crop.png', 'r')
im = im.convert("L")
im

In [ ]:
# data = np.asarray(im)
#you can create images from array like that (rot90) for this one ->
#rotate = Image.fromarray(np.rot90(data), 'L')

### Image Split

If the image is not a multiple of 8, make it one by adding row and cells.
Divide the image into macro-block of 8x8pixels.

In [ ]:
def split(data):
    """
    Split a multiple of 8 image into 8x8 macro block list
    """
    
    blocks = []
    
    for i in range(0, data.shape[0], 8):
        for j in range(0, data.shape[1], 8):
            
            block = [[], [], [], [], [], [], [], []]
            
            for a in range (i, i + 8):
                for b in range (j, j + 8):
                    block[a - i].append(data[a][b])
                    
            blocks.append(block)
    return blocks

In [ ]:
def multiple_8(matrix):
    """
    Make any dimension image into a multiple of 8 by adding rows and columns
    by border pixels mirroring
    """
    
    a, b = matrix.shape
    print('Dimension:', matrix.shape)

    if a % 8 != 0 or b % 8 != 0 :
        print("Need padding...")
    else:
        return matrix

    padding_a = 8 - a % 8
    padding_b = 8 - b % 8

    # If the number of pixels to add on one line or a column, it is problematic
    print("Padding:", padding_a, padding_b)
    if padding_a > a or padding_b > b:
        print("Error: cannot do mirror padding, for now")
        return matrix
    
    matrix = matrix.tolist()
    
    # Mirror the pixels on the lines by copying and reversing the pixel line
    for i in range(a):
        tmp = matrix[i][b - padding_b:].copy()
        tmp.reverse()
        matrix[i] += tmp
    
    # Mirror the border lines starting from the bottom
    for i in range(padding_a):
        matrix.append(matrix[a - 1 - i])
    
    matrix = np.array(matrix)
    print('New Dimension:', matrix.shape)

    return matrix.astype('uint8')

In [ ]:
test = np.arange(1, 127)
test = test.reshape(9, 14)

test = multiple_8(test)
print(test)

In [ ]:
# Load the image into a numpy matrix
data = np.asarray(im)

# Make the matrix dimensions multiples of 8
data = multiple_8(data)
img = Image.fromarray(data, 'L')
display(img)

# Cut into macro-block
macro_blocks = split(data)

In [ ]:
# See a macro-block
len(macro_blocks)
block = macro_blocks[3104]
test_block = np.array(block)
test_macro = Image.fromarray(test_block, 'L')

test_macro = test_macro.resize((200,200), resample=0)
display(test_macro)
block

### Apply DCT matrix

explain here

In [ ]:
#Create DCT matrix for NxN dimensions
def DCT(N):
    
    mat = np.zeros(shape=(N,N))
    
    for i in range (N):
        mat[0,i] = np.sqrt(1/N)
        
    for i in range (1, N):
        for j in range (N):
            mat[i,j] = np.sqrt(2/N) * np.cos(np.pi * (2 * j + 1) * i / (2 * N))
    
    return mat

In [ ]:
D8 = DCT(8)
D8

In [ ]:
macros = np.array(macro_blocks)

Im_DCT = np.round(D8 @ (macros - 128) @ D8.T)

#Look a the same previous macro block
print(Im_DCT[3104])

In [ ]:
# Working like in the slide

course_mat = [[52, 55, 61, 66, 70, 61, 64, 73],
             [63, 59, 55, 90, 109, 85, 69, 72],
             [62, 59, 68, 113, 144, 104, 66, 73],
             [63, 58, 71, 122, 154, 106, 70, 69],
             [67, 61, 68, 104, 126, 88, 68, 70],
             [79, 65, 60, 70, 77, 68, 58, 75],
             [85, 71, 64, 59, 55, 61, 65, 83],
             [87, 79, 69, 68, 65, 76, 78, 94]]

mat_bi = np.array(course_mat)

print(mat_bi)
print()

bi = np.round(D8 @ (mat_bi - 128) @ D8.T)

print(bi)

### Quantification of the matrix

Explain here

In [ ]:
# Create Matrix Quantification

def quantification(q):
    mat = np.zeros(shape=(8,8))
    
    Q_arr = [[16, 11, 10, 16, 24, 40, 51, 61],
         [12, 12, 14, 19, 26, 58, 60, 55],
         [14, 13, 16, 24, 40, 57, 69, 56],
         [14, 17, 22, 29, 51, 87, 80, 62],
         [18, 22, 37, 56, 68, 109, 103, 77],
         [24, 35, 55, 64, 81, 104, 113, 92],
         [49, 64, 78, 87, 103, 121, 120, 101],
         [72, 92, 95, 98, 112, 100, 103, 99]]
    
    Q = np.array(Q_arr)
    
    if q < 50:
        a = 5000/q
    else:
        a = 200 - 2 * q
        
    return np.floor((a * Q + 50 )/ 100)
        

In [ ]:
# Same as in the slide
bi_q = np.round(bi / quantification(50))
bi_q

In [ ]:
# For our real images
q = 50
Im_q = np.round(Im_DCT / quantification(q))

# Always our little 3104 favorite macro-block
Im_q[3104]

### Zigzag

Seeing the face of the matrix, we can deduce that using a "zigzag" method of storing inputs will leads to a lot of 0 at the end that we can throw to reduce the overall data.

In [ ]:
# Zigzag travel of a 8x8 matrix
def zigzag(matrix):
    x, y = 0, 0
    a, b = 7, 7
    direction = (0, 0)
    res = []
    
    while True:
        res.append(matrix[x][y])
        if y == b:
            res.append(matrix[x + 1][y])
            y -= 1
            x += 2
            direction = (1, -1)
        elif x == 0:
            res.append(matrix[x][y + 1])
            x +=1
            direction = (1, -1)
        elif x == a:
            res.append(matrix[x][y + 1])
            if y + 1 == b:
                break
            y += 2
            x -= 1
            direction = (-1, 1)
        elif y == 0:
            res.append(matrix[x + 1][y])
            y += 1
            direction = (-1, 1)
        else:
            x += direction[0]
            y += direction[1]

    return res

In [ ]:
# Remove unnecessary zeros at the end of the list
def zero_removal(l):
    i = len(l) - 1
    while i >= 0 and l[i] == 0:
        i-=1
        
    return l[:i+1]

In [ ]:
# Zigzag travel test
test = np.arange(1, 65)
test = test.reshape(8, 8)

print(test)

zz = zigzag(test)
print(zz)

# Test useless zeros removal
l = [1, 2, 3, 4, 5, 0, 0, 0, 1, 0]
print(zero_removal(l))

In [ ]:
# Like in the slide
ms = zero_removal(zigzag(bi_q))
print(ms)

In [ ]:
# Back to our image
Im_zig = []

for i in range (len(Im_q)):
    Im_zig.append(zero_removal(zigzag(Im_q[i])))

#hello to our little 3104 macro block again !
print(Im_zig[3104])

### Huffman

Huffman is used with optimised tables to compress data into shorter bites

In [ ]:
# TODO huffman

### RGB - YUV 
Explain here